In [1]:
# default_exp inventory

In [2]:
#hide
!pip install -q nbdev lambdasdk s3bz

In [3]:
#hide
import pickle
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-product-master-inventory'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']

# Inventory

> upload and download inventory data from villa master backend

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
#export
from json.decoder import JSONDecodeError
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
import bz2, json, boto3, base64, logging

In [6]:
#export
class Endpoints:
  '''get endpoint names from branch name'''
  def __init__(self, branchName='manual-dev'):
    self.branchName = branchName
  updateWithS3 = lambda self: f'update-inventory-s3-{self.branchName}'
  inputS3 = lambda self: f'input-bucket-{self.branchName}'
  querySingleProduct = lambda self: f'single-product-query-{self.branchName}'
  
  
class InventorySdk:
  ''' interact with villa inventory database '''
  def __init__(self, branchName = 'dev', user = None, pw = None, 
               region = 'ap-southeast-1'):
    self.branchName = branchName
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    self.endpoint = Endpoints(branchName=branchName)
    
    
  def updateWithS3(self, data:dict, 
                   key:str = 'allProducts',
                   invocationType:str = 'Event'):
    
    # save to s3
    S3.save(key = key, 
            objectToSave = data , 
            bucket = self.endpoint.inputS3(),
            user=self.user, pw=self.pw)
    logging.info(f'saving to s3 completed')
    
    lambdaPayload = {
        'inputBucketName': self.endpoint.inputS3(),
        'inputKeyName': key
    }
    logging.info(f'input to lambda is {lambdaPayload}')
    try:
      result = self.lambdaClient.invoke(functionName= self.endpoint.updateWithS3() ,input=lambdaPayload,
                                    invocationType= invocationType )
      if result: return Response.getReturn(result)
    except JSONDecodeError:
      logging.warning('no return from function')
      return True

  def querySingleProduct(self, functionName='single-product-query-dev-manual', 
                         ib_prcode= None, user=None, pw=None):
    '''query a single product'''
    input = { 'ib_prcode': ib_prcode }
    return self.lambdaClient.invoke(
        functionName = functionName, input = input )['Payload'].read()


In [7]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
import boto3


## generate dummy data for testing

In [12]:
%%time
#Dummy Data
numberOfRows = 10000
@dataclass_json
@dataclass
class Inventory:
  ib_prcode:str
  ib_brcode:str
  ib_cf_qty:str
  new_ib_vs_stock_cv:str

sampleLargeRandomInput = [ Inventory.from_dict({
    'ib_brcode' : str(randrange(1000,1030,1)),
    'ib_prcode' : str(randrange(10000,100000,1)),
    'ib_cf_qty' : str(randrange(-10,1000,1)),
    'new_ib_vs_stock_cv' : str(randrange(-10,1000,1))
  }).to_dict() for _ in range(numberOfRows)]
sampleLargeRandomInput[0]

CPU times: user 5.39 s, sys: 268 ms, total: 5.66 s
Wall time: 6.56 s


{'ib_prcode': '42961',
 'ib_brcode': '1016',
 'ib_cf_qty': '596',
 'new_ib_vs_stock_cv': '794'}

## create main object

In [13]:
sdk = InventorySdk(user=USER, pw=PW, branchName = 'dev-manual')

## Upload the batch data using s3

In [16]:
%%time
print(f'uploading {len(sampleLargeRandomInput)} items')
result = sdk.updateWithS3(
    sampleLargeRandomInput,
    invocationType = 'Event'
  )
result

uploading 10000 items


CPU times: user 266 ms, sys: 9.56 ms, total: 276 ms
Wall time: 633 ms


True

In [ ]:
%%time
result = 

In [11]:
#hide
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted inventory.ipynb.
